# Enabling and embedding reverse communication solvers

Julia Computing regularly assists clients in developing custom Julia functionality tailored to their specific application and operational needs.

A recent client engagement involved the following requirements:

* Integrate Julia in a large C/C++ batch processing workflow
* Call Julia as a library from C
* Provide Julian alternatives to existing optimization and root finding solvers
* Wrap client's existing solvers with a Julia interface
* Provide a reverse communication interface to Julia solvers, including the wrapped solvers
  * Objective & derivative functions cannot be passed as arguments to Julia solver (no wrapping of cost function)
  * Objective & derivative functions should be executed in C/C++ 
* Enable future infrastructure for easy migration of algorithms from front office to back office without recoding

## What is the difference between forward and reverse communication solvers?

### Forward communication solvers

Forward communication is the strategy employed by most Julia optimization and root finding packages (Roots.jl, Optim.jl, NLsolve.jl, NLopt.jl, etc.)

In each of these packages, an objective function is passed as one of the arguments to a root finder or optimizer.  

The solver directly invokes the passed in objective function as necessary.

Use of forward communication can allow for incorporating powerful techniques, such as automatic differentiation.

Example:

In [1]:
using Optim, NLsolve

###################################################
# Rosenbrock objective and derivative functions defined for use by Optim.jl

function rosenbrock_optim(x::Vector)
    (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end 

function rosenbrock_optim_gradient!(x::Vector, storage::Vector)
    storage[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    storage[2] = 200.0 * (x[2] - x[1]^2)
end

function rosenbrock_optim_hessian!(x::Vector, storage::Matrix)
    storage[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    storage[1, 2] = -400.0 * x[1]
    storage[2, 1] = -400.0 * x[1]
    storage[2, 2] = 200.0
end

f = rosenbrock_optim
g = rosenbrock_optim_gradient!
h = rosenbrock_optim_hessian!

####################################################
# Rosenbrock objective and derivative functions defined for use by NLsolve.jl

function rosenbrock_nlsolve_f!(x::Vector, fvec::Vector)
    fvec[1] = (1 - x[1])^2
    fvec[2] = 100.0 * (x[2] - x[1]^2)^2
end

function rosenbrock_nlsolve_g!(x::Vector, fjac::Matrix)
    fjac[1,1] = -2.0 * (1 - x[1])
    fjac[1,2] = 0.0
    fjac[2,1] = -400.0 * (x[2] - x[1]^2) * x[1]
    fjac[2,2] = 200.0 * (x[2] - x[1]^2)
end

function rosenbrock_nlsolve_fg!(x::Vector, fvec::Vector, fjac::Matrix)
    fvec[1]   = (1.0 - x[1])^2
    fvec[2]   = 100.0 * (x[2] - x[1]^2)^2
    fjac[1,1] = -2.0 * (1 - x[1])
    fjac[1,2] = 0.0
    fjac[2,1] = -400.0 * (x[2] - x[1]^2) * x[1]
    fjac[2,2] = 200.0 * (x[2] - x[1]^2)
end

f! = rosenbrock_nlsolve_f!
g! = rosenbrock_nlsolve_g!
fg! = rosenbrock_nlsolve_fg!

df = NLsolve.DifferentiableMultivariateFunction(f!, g!, fg!)

;

#### Passing cost/derivative functions as an input arguments

In [2]:
@show results = Optim.optimize(f, [-1.2, 1.0], LBFGS())

@show results = Optim.optimize(f, g, [-1.2, 1.0], LBFGS())

@show results = Optim.optimize(f, g, h, [-1.2, 1.0], Newton())

@show results = NLsolve.nlsolve(f!, [-1.2, 1.0], method = :newton)

@show results = NLsolve.nlsolve(f!, g!, [-1.2, 1.0], method = :trust_region)

@show results = NLsolve.nlsolve(df, [-1.2, 1.0])
;

results = Optim.optimize(f,[-1.2,1.0],LBFGS()) = Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-1.2,1.0]
 * Minimizer: [0.9999999926455753,0.999999985283916]
 * Minimum: 0.000000
 * Iterations: 30
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-08: false
   * |g(x)| < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 128
 * Gradient Calls: 128
results = Optim.optimize(f,g,[-1.2,1.0],LBFGS()) = Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-1.2,1.0]
 * Minimizer: [1.000000000022946,1.000000000043234]
 * Minimum: 0.000000
 * Iterations: 26
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-08: false
   * |g(x)| < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 115
 * Gradient Calls: 115
results = Optim.optimize(f,g,h,[-1.2,1.0],Newton()) = Results of Optimization Algorit

The objective and derivative functions can take required parameters as input arguments, or some parameters can be passed via closures.

The important point is that the forward communication solver directly [invokes the cost function](https://github.com/JuliaOpt/Optim.jl/blob/b01425755c653ff8fe11cc1e3aa41b648331f6dd/src/l_bfgs.jl#L97) within its own internal logic.  

Hence the objective and derivative functions need input arguments structured in a well defined manner for being called directly within the solver.

### Reverse communication solvers

Reverse communication is a classic technique in which objective and derivative functions are not passed directly as inputs to the solver.  

Solver returns to its calliing environment whenever an objective or derivative function value is required.  

After execution of the objective or derivative function in the calling environment, the solver is called, advancing execution from its prior location.

#### Traditional methods of transforming forward communication to reverse communication

Transforming a forward communication solver into a reverse communication solver can be performed by:
* Tracing execution of the forward solver
* Marking the locations of each objective or derivative function evaluation
* Defining a stateful input parameter for storing current objective and derivative function values and location within the solver
* Adding logic to a modified solver to resume execution at desired locations
* Calling the new reverse communication solver repeatedly until reaching an exit condition

Example of trapezoidal integration by forward communication, marking locations of interest for reverse communication

(Adapted from example in ["Writing Scientific Software: A Guide to Good Style"](http://www.cambridge.org/us/academic/subjects/computer-science/scientific-computing-scientific-software/writing-scientific-software-guide-good-style))

#### Trapezoidal integration via foward communication 
```julia
function integrate(f::Function, a, b, n) # f is a function taking one scalar input
    h = (b-a)/n
    # Position 1
    s = 0.5*f(a)
    # Position 2
    s += 0.5*f(b)
    for i = 1:n-1
        # Position 3
        s += f(a+i*h)
    end
    # Position 4
    val = s*h
end

```

#### Reverse communication construction via a mechanical transformation process
```julia
type InternalState
    position::Int
    i::Int
    n::Int
    h::Float64
    v::Float64
    flag::Int
end

# Controller flag argument
# flag = 0   Initialize on entry,           success on exit
# flag = 1    fx = f(x) on entry, Evaluate fx= f(x) on exit
# flag = -1       Error on entry,           failure on exit

function integrate_rc(s::InternalState, x::Ptr{Float64}, fx::Float64, a::Float64, b::Float64, n::Int)
    val = NaN
    if s.flag == 0
        s.position = 1
        s.n = n
        x[] = a
        s.flag = 1
    elseif s.flag == 1
        if s.position == 1
            s.v = 0.5*fx
            x[] = b
            s.flag = 1
            s.position = 2
        elseif s.position == 2
            s.v += 0.5*fx
            s.i = 1
            x[] = a + s.i*s.h
            s.flag = 1
            s.position = 3
        elseif s.position == 3
            s.v += fx
            s.i += 1
            x[] = a + s.i*s.h
            s.flag = 1
            if s.i == n-1
                s.position = 4
            end
        elseif s.position == 4
            s.v += fx
            val = s.v*s.h
            s.flag = 0
        else
            s.flag = -1
        end
        s.flag = -1
    end
    return val
end
```

Performing this type of transformation can be a complicated process.

Especially for solvers that use multiple layers of different functions, files and dependent packages.

## Using Julia tasks to transform an existing forward communication solver

Using Julia coroutines and task scheduling, a forward solver executing as a task can be paused to obtain objective or derivative values in a separate task.

Upon obtaining the desired function value, a task switch can resume the original solver execution.

Structuring this transformation incorporates the following components:

* A task for the main execution environment
  * Objective and derivative functions are evaluated in the scope of this main task
* A task for the solver
* Stub function(s) as input arguments to the forward communication solver
  * Each stub function:
    * Defines a value, or a tuple of values, for passing to the main calling environment
    * Performs a task switch to the main calling environment
* A driver while loop
  * Controls the solver and function execution
  * Checks the state of the solver task
  * Contains logic for execution of the actual objective and derivative functions
  * Performs a task switch to the solver, resuming execution within a stub function

### Reverse communication example using only an objective function

In [3]:
# Define the main task
maintask = current_task()

# Define the stub function
function f_t(x::Vector)
    yieldto(maintask, x)
end

# Define an initial guess
guess = [-1.2; 1.0]

# Create a task for the solver
solver = @task Optim.optimize(f_t, guess, NelderMead())

# Driving loop
next_x = copy(guess)
while !istaskdone(solver)
    fx = f(next_x)                 # Objective function evaluation
    next_x = yieldto(solver, fx)   # Yielding back to the solver task
end
result = next_x

@show result
;

result = Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-1.2,1.0]
 * Minimizer: [1.0000435704852513,1.0000890072130664]
 * Minimum: 0.000000
 * Iterations: 79
 * Convergence: true
   * |x - x'| < NaN: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
   * |g(x)| < NaN: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 149
 * Gradient Calls: 0


### Reverse communication example using objective and derivative functions

In [4]:
# Main task
maintask = current_task()

# Stub functions
# Include a symbol in "vals" for detecting which stub function was called within while loop
function f_t!(x::Vector, fval::Vector)
    vals = (:f, x, fval)
    yieldto(maintask, vals)
end
function g_t!(x::Vector, fjac::Matrix)
    vals = (:g, x, fjac)
    yieldto(maintask, vals)
end
function fg_t!(x::Vector, fval::Vector, fjac::Matrix)
    vals = (:fg, x, fval, fjac)
    yieldto(maintask, vals)
end

# Initial guess and fvec allocations
guess = [-1.2; 1.0]
fval = [Inf; Inf]

df = NLsolve.DifferentiableMultivariateFunction(f_t!, g_t!, fg_t!)

# Create a task for the solver
solver = @task NLsolve.nlsolve(df, guess)

vals = (:f, guess, fval)
while !istaskdone(solver)
    if vals[1] == :f
        guess = vals[2]
        fval  = vals[3]
        f!(guess, fval)              # Evaluate the objective function
        vals = yieldto(solver, vals)
    elseif vals[1] == :g
        guess    = vals[2]
        jacobian = vals[3]
        g!(guess, jacobian)          # Evaluate the derivative function
        vals = yieldto(solver, vals)
    elseif vals[1] == :fg
        guess    = vals[2]
        fval     = vals[3]
        jacobian = vals[4]
        fg!(guess, fval, jacobian)   # Evaluate the combined objective and derivative function
        vals = yieldto(solver, vals)
    end
end

@show vals
;

vals = Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [-1.2,1.0]
 * Zero: [0.9999734745492139,0.9999379198581341]
 * Inf-norm of residuals: 0.000000
 * Iterations: 37
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 38
 * Jacobian Calls (df/dx): 25


## Embedding the reverse communication solver in a C calling environment

Next we replicate the above logic using Julia's C API, allowing for Julia to be embedded in a C application.

In so doing, the objective and derivative functions can be defined as C functions instead of being Julia functions, meeting one of the client's main objectives.

```C
//rc.c

#include <julia.h>
#include <stdio.h>
#include <math.h>

#ifdef _OS_WINDOWS_
__declspec(dllexport) __cdecl
#endif

int r_f(jl_array_t *x, jl_array_t *f)
{
    double* xd = (double*)jl_array_data(x);
    double* fd = (double*)jl_array_data(f);

    fd[0] = pow(1.0 - xd[0], 2.0);
    fd[1] = 100.0 * pow(xd[1] - pow(xd[0], 2.0), 2.0);

    return 0;
}

int r_g(jl_array_t *x, jl_array_t *jac)
{
    double* xd = (double*)jl_array_data(x);
    double* jd = (double*)jl_array_data(jac);
    
    jd[0] = -2.0 * (1 - xd[0]);                         //1,1 element
    jd[1] = -400.0 * (xd[1] - pow(xd[0], 2.0)) * xd[0]; //2,1 element
    jd[2] = 0.0;                                        //1,2 element
    jd[3] = 200.0 * (xd[1] - pow(xd[0], 2.0));          //2,2 element

    return 0;
}

void r_fg(jl_array_t *x, jl_array_t *f, jl_array_t *jac)
{
    double* xd = (double*)jl_array_data(x);
    double* fd = (double*)jl_array_data(f);
    double* jd = (double*)jl_array_data(jac);
    
    fd[0] = pow(1.0 - xd[0], 2.0);
    fd[1] = 100.0 * pow(xd[1] - pow(xd[0], 2.0), 2.0);

    jd[0] = -2.0 * (1 - xd[0]);                         //1,1 element
    jd[1] = -400.0 * (xd[1] - pow(xd[0], 2.0)) * xd[0]; //2,1 element
    jd[2] = 0.0;                                        //1,2 element
    jd[3] = 200.0 * (xd[1] - pow(xd[0], 2.0));          //2,2 element
}

jl_value_t* yieldto(jl_value_t *solver, jl_value_t *vals) {
    jl_function_t *yieldto_func = jl_get_function(jl_base_module, "yieldto");
    return jl_call2(yieldto_func, solver, vals);
}

int istaskdone(jl_value_t *solver)
{
    jl_function_t *taskdone = jl_get_function(jl_base_module, "istaskdone");
    return jl_call1(taskdone, solver) == jl_true;
}

#define __noinline __attribute__((noinline))
void __noinline real_main()
{
    int n = 2;

    // Represent arrays that will be created and owned by C
    double *guess    = (double*)malloc(sizeof(double)*n);
    double *fval     = (double*)malloc(sizeof(double)*n);
    double *jacobian = (double*)calloc(n*n, sizeof(double));

    // Assign initial values into the above arrays
    guess[0] = -1.2;
    guess[1] =  1.0;
    fval[0]  =  1.0/0.0;
    fval[1]  =  1.0/0.0;

    // Root objects with the GC
    jl_value_t **args;
    // args[0] - NLsolve.DifferentiableMultivariateFunction(f!, g!, fg!)
    // args[1] - :f
    // args[2] - :g
    // args[3] - :fg
    // args[4] - array_type
    // args[5] - guess::Vector{Float64}
    // args[6] - fval::Vector{Float64}
    // args[7] - jacobian::Matrix{Float64}
    // args[8] - tuple function
    // args[9] - solver_task function
    
    JL_GC_PUSHARGS(args,10);

    jl_eval_string("using NLsolve");
    jl_eval_string("maintask = current_task()");
    jl_eval_string("f_t!(x::Vector, fvec::Vector) = yieldto(maintask, (:f, x, fvec))");
    jl_eval_string("g_t!(x::Vector, fjac::Matrix) = yieldto(maintask, (:g, x, fjac))");
    jl_eval_string("fg_t!(x::Vector, fvec::Vector, fjac::Matrix) = yieldto(maintask, (:fg, x, fvec, fjac))");
    
    args[0] = jl_eval_string("NLsolve.DifferentiableMultivariateFunction(f_t!, g_t!, fg_t!)");
    args[1] = (jl_value_t*) jl_symbol("f");
    args[2] = (jl_value_t*) jl_symbol("g");
    args[3] = (jl_value_t*) jl_symbol("fg");
    args[4] = (jl_value_t*) jl_apply_array_type(jl_float64_type, 1 );
    // Wrap the C arrays as Julia arrays. The final "0" argument means
    // that Julia does not take ownership of the array data for GC purposes.
    args[5] = (jl_value_t*) jl_ptr_to_array_1d(args[4], guess, n, 0);      
    args[6] = (jl_value_t*) jl_ptr_to_array_1d(args[4], fval, n, 0);       
    args[7] = (jl_value_t*) jl_ptr_to_array_1d(args[4], jacobian, n*n, 0);
    args[8] = (jl_value_t*) jl_get_function(jl_base_module, "tuple");
    args[9] = jl_eval_string("solver_task(df, guess) = @task nlsolve(df, guess)");

    { 
        jl_value_t* vals   = jl_call3((jl_function_t*) args[8], args[1], args[5], args[6]);  // "vals" tuple
        jl_value_t* solver = jl_call2((jl_function_t*) args[9], args[0], args[5]);           // solver task
        
        JL_GC_PUSH2(&vals, &solver);

        jl_yield();
        while (!istaskdone(solver)) {
            if (jl_get_nth_field(vals, 0) == args[1]) {        // :f case
                args[5] = jl_get_nth_field(vals, 1);
                args[6] = jl_get_nth_field(vals, 2);
                r_f((jl_array_t*) args[5], (jl_array_t*) args[6]);
                vals = yieldto(solver, vals);
            } else if (jl_get_nth_field(vals, 0) == args[2]) { // :g case
                args[5] = jl_get_nth_field(vals, 1);
                args[7] = jl_get_nth_field(vals, 2);
                r_g((jl_array_t*) args[5], (jl_array_t*) args[7]); 
                vals = yieldto(solver, vals);
            } else if (jl_get_nth_field(vals, 0) == args[3]) { // :fg case
                args[5] = jl_get_nth_field(vals, 1);
                args[6] = jl_get_nth_field(vals, 2);
                args[7] = jl_get_nth_field(vals, 3);
                r_fg((jl_array_t*) args[5], (jl_array_t*) args[6], (jl_array_t*) args[7]); 
                vals = yieldto(solver, vals);
            }
        }
        jl_show(jl_stderr_obj(), vals);
        jl_eval_string("println(\"\n\")");
        JL_GC_POP();
    }
    JL_GC_POP();
    
    free(guess);
    free(fval);
    free(jacobian);
}

void __noinline real_main_wrapper()
{
    real_main();
}

int main()
{
    jl_init(NULL);

    real_main(); // this should be a separate function from main()

    int ret = 0;
    jl_atexit_hook(ret);
    return ret;
}
```

Example gcc compilation

```bash
$ export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/julia-0.4.5/lib/julia
$ export JULIA_HOME=/opt/julia-0.4.5/bin
$ export JULIA_DIR=/opt/julia-0.4.5 
$ gcc -o rc -fPIC -I$JULIA_DIR/include/julia -L$JULIA_DIR/lib/julia rc.c -ljulia $JULIA_DIR/lib/julia/libstdc++.so.6 -lm
```

Also see the [Embedding Julia documentation](http://docs.julialang.org/en/release-0.4/manual/embedding/) as well as the [embedding.c](https://github.com/JuliaLang/julia/blob/master/examples/embedding.c) and [Makefile](https://github.com/JuliaLang/julia/blob/master/examples/Makefile) examples in the Julia source code.